In [2]:
import datetime
import requests
import pandas as pd
import pandas_datareader.data as web
from bcb import sgs
from zipfile import ZipFile


In [3]:
# Importing matplotlib and setting aesthetics for plotting later.
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
plt.style.use('fivethirtyeight')

# FIIs e pandemia de Covid-19

## FIIs de Shopping

In [4]:
start = pd.to_datetime(['2020-02-15']).astype(int)[0]//10**9 # convert to unix timestamp.
end = pd.to_datetime(['2022-10-18']).astype(int)[0]//10**9 # convert to unix timestamp.

In [ ]:
tickers = ['VISC11.SA', 'ABCP11.SA', 'HGBS11.SA']

shoppings = pd.DataFrame()
for fii in tickers:
    url = 'https://query1.finance.yahoo.com/v7/finance/download/' + fii + '?period1=' + str(start) + '&period2=' + str(end) + '&interval=1d&events=history'
    shoppings[fii] = pd.read_csv(url)['Close']

shoppings.head()

In [ ]:
def plot_normalized_returns(b3_data):
    log_returns = b3_data / b3_data.iloc[0]*100
    log_returns.plot(figsize=(10,5))
    plt.ylabel('Preço')
    plt.xlabel('Dias')
    plt.show()
    
def plot_returns(b3_data):
    log_returns = b3_data
    log_returns.plot(figsize=(10,5))
    plt.ylabel('Preço')
    plt.xlabel('Dias')
    plt.show()

In [ ]:
plot_normalized_returns(shoppings) 

In [ ]:
plot_returns(shoppings)

## FIIs de Logística

In [ ]:
tickers = ["HGLG11.SA", "GGRC11.SA", "FIIB11.SA", "ALZR11.SA", "LVBI11.SA"]

logistica = pd.DataFrame()
for fii in tickers:
    url = 'https://query1.finance.yahoo.com/v7/finance/download/' + fii + '?period1=' + str(start) + '&period2=' + str(end) + '&interval=1d&events=history'
    logistica[fii] = pd.read_csv(url)['Close']

logistica.head()

In [ ]:
plot_normalized_returns(logistica) 

In [ ]:
plot_returns(logistica)

### Evolução do HGRU11 (2020-2022)

In [ ]:
for year in [2020]:
    url_dados = f"https://dados.cvm.gov.br/dados/FII/DOC/INF_ANUAL/DADOS/inf_anual_fii_{year}.zip"
    r = requests.get(url_dados)
    nome_arquivo = f"dados_{year}.zip"
    open(nome_arquivo, 'wb').write(r.content)

    with ZipFile(f"dados_{year}.zip", 'r') as zip:
        zip.extractall("dados_cvm")
        dados_cvm = pd.read_csv(f"dados_cvm/inf_anual_fii_geral_{year}.csv", error_bad_lines=False, encoding= 'unicode_escape')
        

dados_cvm.head()

### SELIC

In [ ]:
selic = sgs.get(('selic', 432), start = '2020-02-15')

In [ ]:
selic.plot(figsize = (10, 5))
plt.title("Taxa de juros - Meta Selic definida pelo Copom - %a.a")

### IPCA

In [ ]:
ipca = sgs.get(('IPCA', 433), start='2020-01-01')
ipca_anualizado = ipca.rolling(12)
ipca_acumulado_em_12 = ipca_anualizado.apply(lambda x: (1 + x/100).prod() - 1).dropna() * 100


ipca_acumulado_em_12.plot(figsize=(12,6))
plt.title('Fonte: https://dadosabertos.bcb.gov.br', fontsize=10)
plt.suptitle('IPCA acumulado 12 meses - Janela Móvel', fontsize=18)
plt.xlabel('Data')
plt.ylabel('%')
plt.legend().set_visible(False)